# Imports 

In [1]:
import pandas as pd
import numpy as np
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Read in data and make sure relevant columns are string/character 

- San Diego data: `naics_code` and `account_key`
- NAICS details data: `naics` 

Run code below; if pulling from github, pathname should be fine; if working elsewhere may need to edit path name at read in 

In [2]:
# uncomment if u need to find your path: os.getcwd()
SD = sd_df = pd.read_csv("../public_data/sd_df.csv")
NA = naics_df = pd.read_csv("../public_data/naics_df.csv")

In [11]:
cols_sd_use = ["naics_code", "account_key"]
cols_naics_use = ["naics"]

sd_df[cols_sd_use] = sd_df[cols_sd_use].astype(str)
naics_df[cols_naics_use] = naics_df[cols_naics_use].astype(str)

sd_df.dtypes
naics_df.dtypes
sd_df.head()
naics_df.head()
sd_df.size

account_key          object
dba_name             object
council_district     object
naics_code           object
naics_description    object
naics_nchar           int64
dtype: object

naics                object
naics_description    object
dtype: object

,account_key,dba_name,council_district,naics_code,naics_description,naics_nchar
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4


,naics,naics_description
0,111140,Wheat Farming
1,111160,Rice Farming
2,111150,Corn Farming
3,111110,Soybean Farming
4,111120,Oilseed (except Soybean) Farming


2118

## "Inner join"- retain only San Diego businesses with details on their NAICS code

- Use the `naics_code` column in the San Diego business data as the join key
- Use the `naics` column in the NAICS code details data as the join key

- Do an inner join of the San Diego data onto the NAICS code details using these join keys
- After the inner join, print some examples of San Diego businesses lost in the merge
- Use value_counts() on the `naics_nchar` column in the San Diego data to see why they might have gotten lost


In [20]:
df = pd.merge(sd_df, naics_df, left_on = sd_df["naics_code"], right_on = naics_df["naics"], how = "inner")

lost = sd_df[~sd_df["naics_code"].isin(df["naics_code"])]
lost.reset_index(drop = True)

df["naics_nchar"].value_counts()

,account_key,dba_name,council_district,naics_code,naics_description,naics_nchar
0,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4
1,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4
2,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4
3,1979053082,GRIMM VRANJES GREER STEPHAN & BRIDGMAN LLP,cd_3,5411,LEGAL SERVICES,4
4,1981000840,BENINK & SLAVENS LLP,cd_7,5411,LEGAL SERVICES,4
...,...,...,...,...,...,...
313,2006015404,9625 TOWN CENTRE PARTNERS LP,cd_3,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3
314,2007015303,WFP-GATEWAY LP,cd_3,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3
315,2007015336,WFP-MIRAMAR LP,cd_3,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3
316,2010010820,ROSSLYN LOFTS HOUSING PARTNERS LP THE,cd_6,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3


naics_nchar
6    52
Name: count, dtype: int64

## "Left join"- retain all sd businesses even if naics code isn't in `naics_df`

- Using the same join keys as above, and treating the San Diego businesses as the left hand side data, left join the naics code details onto the San Diego businesses
- Use the `indicator` argument within merge to create an indicator, `naics_merge_status`, to help with later merge diagnostics and examine sample of ones that didn't merge
- Use the `suffixes` argument within merge to add `_sd` as the left suffix, `_census` as the right suffix
- Use `naics_merge_status` in the merged result to look at a sample of San Diego businesses that weren't matched with `naics_df`

In [21]:
# your code here

df1 = pd.merge(sd_df, naics_df, left_on = sd_df["naics_code"], right_on = naics_df["naics"], how = "left", indicator='naics_merge_status', suffixes=('_sd', '_census'))

df1.reset_index()

unmatched_businesses = df1[df1['naics_merge_status'] != 'both']
unmatched_businesses.sample(5).reset_index()

,index,key_0,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics,naics_description_census,naics_merge_status
0,0,541211,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,Offices of Certified Public Accountants,both
1,1,5411,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only
2,2,541211,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,Offices of Certified Public Accountants,both
3,3,5411,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only
4,4,5412,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...
365,365,541,2010010820,ROSSLYN LOFTS HOUSING PARTNERS LP THE,cd_6,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3,NaN,NaN,left_only
366,366,811198,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,All Other Automotive Repair and Maintenance,both
367,367,811198,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,All Other Automotive Repair and Maintenance,both
368,368,811198,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,All Other Automotive Repair and Maintenance,both


,index,key_0,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics,naics_description_census,naics_merge_status
0,133,532,2007027081,PACIFIC REALTY ADVISORS LP,cd_7,532,RENTAL & LEASING SERVICES,3,NaN,NaN,left_only
1,358,5411,2022000180,CALIFORNIA LAW PARTNERS LLP,cd_2,5411,LEGAL SERVICES,4,NaN,NaN,left_only
2,296,52312,2020006209,EDWARD D JONES & CO L P,cd_7,52312,SECURITIES BROKERAGE,5,NaN,NaN,left_only
3,85,54111,2005012390,CLERKIN SINCLAIR & MAHFOUZ LLP,cd_3,54111,OFFICES OF LAWYERS,5,NaN,NaN,left_only
4,325,72,2020011305,PRIMOS,cd_1,72,ACCOMMODATION & FOOD SERVICES,2,NaN,NaN,left_only


## Use group by and agg to see if there are differences in merge rates by area

- Using the left-joined dataframe created in previous step, create a boolean indicator, `is_lost`, that equals `True` if the merge indicator is equal to "left_only" (and is otherwise false)
- Find the proportion of businesses by council district that were lost in the left join (SD businesses that failed to match to `naics_df`). To do this, group by `council_district` and use the shortcut of taking the mean of the `is_lost` indicator to find the proportions lost by `council_district`. 

In [19]:
# your code here

df1['is_lost'] = df1['naics_merge_status'] == 'left_only'

proportion_lost_by_district = df1.groupby('council_district')['is_lost'].mean()
print(proportion_lost_by_district)

council_district
cd_1    0.880597
cd_2    0.804348
cd_3    0.812500
cd_4    1.000000
cd_5    0.916667
cd_6    0.859649
cd_7    0.918919
cd_8    0.933333
cd_9    1.000000
Name: is_lost, dtype: float64


## Optional challenge exercise: add lagging 0's and see if merge rate improves

You noticed earlier that a big reason for non-matches is that the San Diego tax certificate NAICS codes were often less than six digits long, while the Census ones were always 6 digits.

You wonder if this is an issue where 0's in some of the SD's data naics codes got cut off (eg 540000 became 54), and if so whether adding these lagging zeros would improve the merge rate in a left join.

- Pad the `naics_code` column in `sd_df` with 0's to get that column up to 6-digits, using one of two approaches: 
  1. `str.pad` in pandas (https://pandas.pydata.org/docs/reference/api/pandas.Series.str.pad.html)
  2. for more of a challenge, write your own function! It should check the # of digits in the naics code string and pad it with 0's at the end up to 6 digits. To execute your function, use row-wise apply: `df.apply(lambda row: funcname(row.column), axis=1)`.
- Perform the same left join as above and see how the match rate changes
- Create an indicator variable--`is_new_match`---for new matches under the padded NAICS code; compare the `naics_description` column from San Diego versus Census in the new dataset for a sample of these new matches and comment on whether the padding seems to be correct

In [23]:
# your code here

def pad_zeros(code):
    while len(code) < 6:
        code = str(code) + "0"
    return code
SD['padded_naics_code'] = SD['naics_code'].apply(pad_zeros)
left_SD_new = SD.merge(NA, how = "left", right_on = "naics", left_on = "padded_naics_code", suffixes = ("_sd", "_census"),
                         indicator = "naics_merge_status")
print(df1.naics_merge_status.value_counts(), left_SD_new.naics_merge_status.value_counts())

naics_merge_status
left_only     318
both           52
right_only      0
Name: count, dtype: int64 naics_merge_status
both          237
left_only     209
right_only      0
Name: count, dtype: int64
